### Data Load & Setting

In [1]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT
import ast

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_

In [85]:
data=pd.read_csv('../data/menu_v3_sample_sentence.csv')

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)

In [86]:
data["org_menu_dict"] = data["org_menu_dict"].apply(ast.literal_eval)

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   org_menu_dict          40 non-null     object
 4   menu_sentence          40 non-null     object
dtypes: object(5)
memory usage: 1.7+ KB


In [60]:
data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence,input_text
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...,잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 맵 .. 아...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들...",미나리 같이 오리 주물럭 아주 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 .. 도...
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있...",토시 갈비 섯구이 구성 소고기 세트 주문 고기 정말 맛있 사이드 주문 쫄면 된장찌개...
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁...",김찌 맵기 조절 가능 칼칼하 제육 짭졸 퐁실퐁실 진짜 부드럽 쌉꿀마 김치찌개 계란말...
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양...",벌교 꼬막 38000 인분 정도 굴회 무침 30 000 재미 무침 양념 상큼 음식 ...


### Functions

In [3]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith("N") or pos.startswith("SL")):
            results.append(token)
    return results

# 명사 추출 함수
def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results

# 명사 추출 함수
def extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (
            len(token) != 1
            and pos.startswith("N")
            or pos.startswith("SL")
            or pos.startswith("V")
        ):
            results.append(token)
    return results

In [8]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
# data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인

### KeyBERT to DataFrame (by. Moonsoo)

In [9]:
# KeyBERT 로드. (KoBERT 사용)

model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [52]:
# keybert돌리고 키워드 리턴하는 함수 (!! -- ngram은 무조건 켜야함 -- !!)
def extract_keywords_from_reviews_candy(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,3),  # 단어 n-gram 범위
        stop_words=None,  # 불용어F
        # use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        # highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드
    # print(f"Keywords: {keywords}\n")

    return keywords

# 데이터 형식을 input text에 맞게 변환하는 함수 제작 (for문 안에서 돌던거를 위로 뺐음)
def doc_to_input_text(review_doc):
    # 식당의 문장 리스트 추출
    input_text = ".".join(review_doc)
    input_text = adverb_remover(input_text)
    input_text=" ".join(input_text)

    return input_text
    

In [87]:
# input text 열 만들기
data['input_text'] = data['menu_sentence'].apply(doc_to_input_text)

In [88]:
# 데이터 생긴거 확인 가능

data.head()

,rst_name,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence,input_text
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"[, 족발 하 고, 불 족발 새우젓 마늘 쌈장 쌈 야채, 실속 알뜰 보쌈 겉절이, ...","[{' 왕족발+막국수)': ['', '왕족발막국수)', ' 왕족발 막국수)', ' ...",[잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면...,잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 맵 .. 아...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[뼈탕, 양념 주물럭, 모듬한마리, 양념주물럭, 오리 주물럭, 훈제오리, 모듬 한 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 고기시키면 뼈탕이 서비스였는데 들...",미나리 같이 오리 주물럭 아주 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 .. 도...
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[토시 살, 된장찌개 밥, 가브리 살, 소고기 세트, 된장찌개밥, 돼지세트 , 김치...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[토시살 갈비살 버 섯구이 구성의 소고기 세트 주문했어요, 고기도 정말 맛있...",토시 갈비 섯구이 구성 소고기 세트 주문 고기 정말 맛있 사이드 주문 쫄면 된장찌개...
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","[왕 계 이 란 말 이, 고기반김치찌개세트, 고기 반 김치찌개 세트, 왕계란말이, ...","[{'소불고기': ['소 불고기', '소불고기']}, {'고추장제육': ['고추장제...","[김찌는 맵기조절이가능하여 칼칼하게먹기딱좋고 제육은 불향이나서 짭졸하니 좋아요, 퐁...",김찌 맵기 조절 가능 칼칼하 제육 짭졸 퐁실퐁실 진짜 부드럽 쌉꿀마 김치찌개 계란말...
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","[물회, 새꼬막양념, 벌교참꼬막, 간 재미, 새 꼬막, 참 꼬막 양념, 간재미 회무...","[{'쭈꾸미구이': ['쭈꾸미 구이', '쭈꾸미구이']}, {'짱뚱어탕': ['짱뚱...","[벌교참꼬막 38000 2인분 정도 될듯해요, 굴회무침 30 000 간재미무침과 양...",벌교 꼬막 38000 인분 정도 굴회 무침 30 000 재미 무침 양념 상큼 음식 ...


In [89]:
from collections import defaultdict

# df1의 각 main_menu가 어떤 카테고리에 속하는지 확인
def sub_to_org(menu, submenu_to_menu):
    for key, org_menuname in submenu_to_menu.items():
        if menu == key:
            return org_menuname
    
    return menu  # 카테고리를 찾지 못한 경우


def duplicate_removed_list(duplicated_list):
    # [[menu1, score1], [menu1, score2]]
    # 각 메뉴 항목에 대한 스코어를 저장할 defaultdict 생성
    menu_scores = defaultdict(list)

    # 데이터를 defaultdict에 저장하여 그룹화
    for menu, score in duplicated_list:
        menu_scores[menu].append(score)

    # 각 메뉴 항목의 스코어 평균을 계산
    averaged_scores = []
    for menu, scores in menu_scores.items():
        average_score = sum(scores) / len(scores)
        averaged_scores.append([menu, average_score])
    # [[menu1, avg_score1], [menu2, avg_score2]]
    
    return averaged_scores


In [110]:
rst_name_list, menu_name_list, keybert_scores = [], [], []
cols = ["rst_name", "main_menu", "keybert_score", "reviews" ,"org_menu_dict"]
seulbeen_output=[]
for i in range(40):
    # 식당의 문장 리스트 추출
    
    input_text = data.loc[i, 'input_text']
    menu_candidates=data.loc[i, 'menu_name_split']
    org_menu_dict=data.loc[i,'org_menu_dict']

    review = data.loc[i,'review_sentence_split']
    rst_name = data.loc[i, 'rst_name']

    results = extract_keywords_from_reviews_candy(input_text,menu_candidates)
    # print(results)

    # 메뉴 이름을 키로 하고 카테고리를 값으로 하는 새로운 딕셔너리 생성
    submenu_to_menu = {}
    for menu_dict in org_menu_dict:
        for key, values in menu_dict.items():
            for value in values:
                submenu_to_menu[value] = key

    menu_score = []
    if results:
        # 각 결과에 대해 df에 들어갈 각 행으로 변환
        for result in results:
            name, score = result[0], result[1]
            name = sub_to_org(name, submenu_to_menu)
            menu_score.append([name, score])
            
        # 같은 원본 메뉴 이름을 가진 것들의 중복 제거
        menu_score = duplicate_removed_list(menu_score)
        for menu, score in menu_score:
            print((menu, score), end=", ")
            rst_info = []

            rst_info.append(rst_name)
            rst_info.append(menu)
            rst_info.append(score)
            rst_info.append(review)
            rst_info.append(org_menu_dict)

            seulbeen_output.append(rst_info)

        del [[rst_info]]
        
    else:
        print(f"PASS")

PASS
('오리주물럭', 0.4592), ('양념주물럭', 0.4182), ('뼈탕', 0.163), ('오겹살', 0.5143), ('목살', 0.4756), ('매콤 쫄면', 0.4726), ('김치찌개+밥', 0.4596), ('가브리살', 0.4242), ('고공세트', 0.3255), ('소고기세트', 0.3055), ('돼지세트 ', 0.2996), ('된장찌개+밥', 0.281), ('제육 2인 세트', 0.4742), ('간장제육', 0.3241), ('쭈꾸미구이', 0.7002), ('짱뚱어탕', 0.6895), ('물회', 0.3199), ('야채곱창', 0.4808), ('곱창', 0.4687), 

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


('미나리 육전', 0.604), ('양지 갈비탕', 0.55055), ('양지 갈비 쌀국수', 0.51325), ('얼큰 양지 소고기 국밥', 0.3777), ('얼큰 스지 볶음', 0.3879), ('양지쌀국수', 0.3701), ('모둠 쌀국수', 0.3679), ('모둠 수육전골', 0.187), ('빠네파스타+파스타+에이드2', 0.603), ('까르보나라', 0.4862), ('마라크림', 0.4429), ('치킨크림리조또', 0.314), ('치즈오븐스파게티', 0.3117), ('파스타+파스타+음료2', 0.2989), ('목살스테이크', 0.2315), ('수육 중', 0.2057), ('내장곰탕', 0.5306), ('도가니탕', 0.4795), ('설렁탕', 0.4458), ('도가니수육', 0.3726), ('갈비탕', 0.3591), ('꼬리곰탕', 0.3461), ('김치전', 0.6207), ('칼제비', 0.4423), ('파전', 0.3493), ('도토리묵', 0.3296), ('항아리수제비', 0.2949), ('콩국수', 0.2845), ('민속국시', 0.2439), ('바지락칼국수', 0.3495), ('소고기 샤브샤브 월남쌈 주말/공휴일', 0.7158), ('한우 샤브샤브 월남쌈 주말 공휴일', 0.5952500000000001), ('소고기+해물샤브샤브월남쌈 주말/공휴일', 0.4156), ('소고기+해물 샤브샤브 월남쌈 평일', 0.3982), ('보배짜장면', 0.6314), ('새우고추짬뽕', 0.5431), ('해장 수재비 짬뽕', 0.4792), ('보배짬뽕', 0.3584), ('크림짬뽕', 0.3143), ('해물쟁반짜장', 0.3012), ('소고기짬뽕', 0.2737), ('해물짬뽕', 0.2545), PASS
('시오버터라멘', 0.5461), ('산쪼메라멘', 0.5103), ('츠케멘', 0.4586), ('카라카라 돈코츠라멘', 0.3768), ('돈코츠라멘', 0.3755), ('냉라멘', 

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


('목살', 0.5021), ('삼겹살', 0.4045), ('양념통닭', 0.1222), ('닭똥집', 0.1063), ('1인분 추가', 0.1001), ('바샤샥 해물전', 0.56255), ('꼬막 소면', 0.4992), ('감자전', 0.6118), ('막국수 물/비빔', 0.4557), ('수육', 0.2438), 

...

('새우튀김', 0.5833), ('서더리탕', 0.4078), ('참돔 )', 0.4006), ('세트 A)', 0.3473), ('해산물 3가지 선택', 0.3159), PASS
('고사리 표고 솥밥', 0.4999), ('한우 차돌박이 삼합', 0.53245), ('미나리 꼬막 무침', 0.5146333333333334), ('차돌박이 비빔냉면', 0.5439), ('곤드레 토마토 바질 솥밥', 0.29769999999999996), ('매콤 우삼겹 숙주 볶음', 0.2982), ('고등어조림', 0.54715), ('고등어구이', 0.492), ('성게미역국', 0.4023), ('세트메뉴', 0.3287), ('한우 곱창구이', 0.5989), ('곱창전골', 0.54965), ('지존짬뽕', 0.638), ('지존짜장', 0.6132), ('수제비 짬뽕', 0.5492), ('찹쌀탕수육 중', 0.5412), ('게살볶음밥', 0.5086), ('순두부 짬뽕밥', 0.5026), ('쌀국수 짬뽕', 0.4825), ('사천탕수육 중', 0.3432), ('스페셜 모듬구이', 0.353), 

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


('생신잔치 한정식 코스', 0.3986), ('상견례 한정식 코스', 0.3976), ('한정식 별코스', 0.2964), ('뿌팟퐁 커리 덮밥', 0.397875), ('뿌 팟퐁 커리', 0.4702), ('쁠라텃', 0.4501), ('무텃', 0.2552), ('태국쌀국수', 0.236), ('똠양쌀국수', 0.1652), ('최루탄주먹밥', 0.3437), 

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


('한우 세트', 0.38675000000000004), ('짜장면', 0.7136), ('쟁반짜장', 0.5853), ('탕수육', 0.5485), ('해물짬뽕', 0.4478), 

In [112]:
seulbeen_output_df=pd.DataFrame(seulbeen_output,columns=cols)

In [113]:
len(seulbeen_output_df)

141

In [116]:
seulbeen_output_df[:30]

,rst_name,main_menu,keybert_score,reviews,org_menu_dict
0,강나루 유황오리주물럭 본점,오리주물럭,0.45920,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
1,강나루 유황오리주물럭 본점,양념주물럭,0.41820,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
2,강나루 유황오리주물럭 본점,뼈탕,0.16300,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모..."
3,고공 구의점,오겹살,0.51430,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
4,고공 구의점,목살,0.47560,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
5,고공 구의점,매콤 쫄면,0.47260,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
6,고공 구의점,김치찌개+밥,0.45960,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
7,고공 구의점,가브리살,0.42420,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
8,고공 구의점,고공세트,0.32550,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."
9,고공 구의점,소고기세트,0.30550,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ..."


In [ ]:
seulbeen_output_df

In [117]:
seulbeen_output_df.to_csv("../data/rst_menu_keybert_score_v2.0.0.csv",encoding='utf-8-sig')

### 감성분석 점수 (by. Taeho)

org_menu_dict에서 main_menu를 포함하는 딕셔너리만 추출
-> filtered_org_menu_dict 생성

filtered_org_menu_dict의 value를 포함하는 문장만 추출
-> filtered_reviews 생성

filtered_reviews의 각 문장들 감성점수 평균 계산 (긍정:양수 / 부정:음수)
-> senti_score 추출

In [118]:
# import library
import torch
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

data = seulbeen_output_df
# data.head()

In [122]:
# main menu로 filtered_org_menu_dict 생성
def filter_org_menu_dict(row):
    # 문자열로 표기된 딕셔너리 실제 딕셔너리 리스트로 변환
    try:
        org_menu_dicts = ast.literal_eval(row['org_menu_dict'])
    except:
        org_menu_dicts = row['org_menu_dict']
    main_menu = row['main_menu']

    # value에 main_menu 포함하는 org_menu_dicts만 필터링
    filtered_org_menu_dicts = []
    for menu_dict in org_menu_dicts:
        for key, variations in menu_dict.items():
            if main_menu in variations:
                filtered_org_menu_dicts.append(menu_dict)
                break
    
    return filtered_org_menu_dicts


# filtered_org_menu_dict로 filtered_reviews 생성
def filter_reviews_by_menu(row):
    # 리뷰 문자열을 리스트로 변환
    reviews = ast.literal_eval(row['reviews'])
    
    # filtered_org_menu_dict로부터 추출된 모든 메뉴(value) 추출
    menu_variations = set()
    for menu_dict in row['filtered_org_menu_dict']:
        for variations in menu_dict.values():
            menu_variations.update(variations)
    
    # menu_variations 포함하는 리뷰만 필터링

    filtered_reviews = [review for review in reviews if any(menu_variation in review for menu_variation in menu_variations)]
    
    return filtered_reviews

# filter_org_menu_dict 적용
data['filtered_org_menu_dict'] = data.apply(filter_org_menu_dict, axis=1)

# filter_reviews_by_menu 적용
data['filtered_reviews'] = data.apply(filter_reviews_by_menu, axis=1)

# 결과 출력
data[['main_menu', 'org_menu_dict', 'filtered_org_menu_dict', 'reviews', 'filtered_reviews']].head()


,main_menu,org_menu_dict,filtered_org_menu_dict,reviews,filtered_reviews
0,오리주물럭,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 찌니가좋아하는 오리주물럭 정말 맛..."
1,양념주물럭,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'양념주물럭': ['양념 주물럭', '양념주물럭']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...",[양념주물럭 정말 맛있었어요]
2,뼈탕,"[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'뼈탕': ['뼈탕', '뼈 탕']}]","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 ., 다 왕맛도리 부..."
3,오겹살,"[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[{'오겹살': ['오 겹 살', '오겹살']}]","['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...",[오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 또 ...
4,목살,"[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[{'목살': ['목살']}],"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[그 중 목살이 태어나서 먹어본 목살 중 제일 맛있었어요, 목살 가브리살 오겹살..."


In [123]:
# load model
tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MOONSOO\.cache\huggingface\hub\models--jaehyeong--koelectra-base-v3-generalized-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\MOONSOO\AppData\Local\Programs\Python\

In [124]:
# filtered_reviews로 senti_score 계산

# 감성분석 함수 (0~1 사이 값으로 출력)
def get_sentiment_score(sentence):
    result = sentiment_classifier(sentence)[0]
    return result['score'] if result['label'] == '1' else 1-(result['score'])

'''
# 여러 문장의 평균 감성 점수 계산 함수
def get_average_sentiment_score(sentences):
    scores = [get_sentiment_score(sentence) for sentence in tqdm(sentences, desc="Processing sentences")]
    return sum(scores) / len(scores) if scores else 0

# 긍정 문장 비율 계산 함수
def get_positive_sentiment_proportion(sentences):
    positive_count = sum(1 for sentence in sentences if sentiment_classifier(sentence)[0]['label'] == '1')
    return positive_count / len(sentences) if sentences else 0
'''


# BPP(BP Penalty) 계산 함수

'''
# 먼저 alpha(우리 표현으로는 lambda) 지정해줘야 함
alpha = 1
나중에 하는 버전으로 코드 변경함
'''

def calculate_BP(sentences):
    positive_count = sum(1 for sentence in sentences if sentiment_classifier(sentence)[0]['label'] == '1')
    total_reviews = len(sentences)
    if total_reviews == 0:
        return 0  # 예외 처리: 리뷰가 없는 경우
    exponent_value = 1 - ((1 + total_reviews) / (1 + positive_count))
    # return (np.e + alpha) ** exponent_value
    return exponent_value


# 'reviews' 열의 문장 리스트를 사용하여 평균 감성 점수 계산
# data['senti_score'] = data['filtered_reviews'].apply(get_average_sentiment_score)
# data['senti_portion'] = data['filtered_reviews'].apply(get_positive_sentiment_proportion)
data['exponent_value'] = data['filtered_reviews'].apply(calculate_BP)

In [125]:
data.head()

,rst_name,main_menu,keybert_score,reviews,org_menu_dict,filtered_org_menu_dict,filtered_reviews,exponent_value
0,강나루 유황오리주물럭 본점,오리주물럭,0.4592,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}]","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 찌니가좋아하는 오리주물럭 정말 맛...",0.000000
1,강나루 유황오리주물럭 본점,양념주물럭,0.4182,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'양념주물럭': ['양념 주물럭', '양념주물럭']}]",[양념주물럭 정말 맛있었어요],0.000000
2,강나루 유황오리주물럭 본점,뼈탕,0.1630,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'뼈탕': ['뼈탕', '뼈 탕']}]","[고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 ., 다 왕맛도리 부...",0.000000
3,고공 구의점,오겹살,0.5143,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[{'오겹살': ['오 겹 살', '오겹살']}]",[오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 또 ...,-0.111111
4,고공 구의점,목살,0.4756,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[{'목살': ['목살']}],"[그 중 목살이 태어나서 먹어본 목살 중 제일 맛있었어요, 목살 가브리살 오겹살...",-0.111111


### Total Score 계산 및 출력 테스트 (by. Taeho)

In [126]:
# total_score 열 계산

alpha = 1.2 # 알파 적용은 아래 두 식 중 양자택일

# alpha 반영하여 e^(alpha*exponent_value) 로 계산하는 식
data['Sentiment_Penalty'] = np.exp(data['exponent_value']*alpha)

data['total_score'] = data['keybert_score'] * data['Sentiment_Penalty']

# rst_name 열의 값이 같은 식당 별로 total_score가 높은 순으로 정렬
sorted_data = data.sort_values(by=['rst_name', 'total_score'], ascending=[True, False])

In [127]:
# 전체 음식점 보기
sorted_data.head()

,rst_name,main_menu,keybert_score,reviews,org_menu_dict,filtered_org_menu_dict,filtered_reviews,exponent_value,Sentiment_Penalty,total_score
0,강나루 유황오리주물럭 본점,오리주물럭,0.4592,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}]","[미나리랑 같이 먹는 오리주물럭 아주 맛있었어요, 찌니가좋아하는 오리주물럭 정말 맛...",0.000000,1.000000,0.459200
1,강나루 유황오리주물럭 본점,양념주물럭,0.4182,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'양념주물럭': ['양념 주물럭', '양념주물럭']}]",[양념주물럭 정말 맛있었어요],0.000000,1.000000,0.418200
2,강나루 유황오리주물럭 본점,뼈탕,0.1630,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","[{'오리주물럭': ['오리 주물럭', '오리주물럭']}, {'모듬한마리': ['모...","[{'뼈탕': ['뼈탕', '뼈 탕']}]","[고기시키면 뼈탕이 서비스였는데 들깨향나는 국물이 맛있었습니다 ., 다 왕맛도리 부...",0.000000,1.000000,0.163000
6,고공 구의점,김치찌개+밥,0.4596,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...",[],[],0.000000,1.000000,0.459600
3,고공 구의점,오겹살,0.5143,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","[{'고공세트': ['고공세트', '고공 세트']}, {'소고기세트': ['소고기 ...","[{'오겹살': ['오 겹 살', '오겹살']}]",[오겹살 진짜 맛도리 기본찬도 정갈하고 김치 하나하나 맛있어서 친구랑 다음에 또 ...,-0.111111,0.875173,0.450102


In [128]:
# 특정 음식점 보기
sorted_data[sorted_data['rst_name'] == '명가옥']

# 명가옥, 짬뽕지존 아차산역점, 보배반점 구의점, 고공 구의점, 라라면옥, 고공 구의점 결과 잘나옴
# 산쪼메 구의점 이슈

,rst_name,main_menu,keybert_score,reviews,org_menu_dict,filtered_org_menu_dict,filtered_reviews,exponent_value,Sentiment_Penalty,total_score
37,명가옥,설렁탕,0.4458,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...",[{'설렁탕': ['설렁탕']}],"[설렁탕 맛있게 잘 먹고 화장실 들렸다, 특설렁탕인데 도가니가 들어가있어요, 설...",-0.108108,0.878333,0.391561
36,명가옥,도가니탕,0.4795,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...",[{'도가니탕': ['도가니탕']}],"[도가니탕포장한건 안비밀 ., 갈비탕1 도가니탕1 깔끔하고 너무맛있어요, 깔끔...",-0.400000,0.618783,0.296707
39,명가옥,갈비탕,0.3591,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...",[{'갈비탕': ['갈비탕']}],"[갈비탕 맛있어요., 갈비탕 고기가득 ㅋㅋㅋ., 갈비탕 먹으면서 혼술 편히 잘 ...",-0.318182,0.682619,0.245129
38,명가옥,도가니수육,0.3726,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...","[{'도가니수육': ['도가니수육', '도가니 수육']}]","[다른 분들은 단체로 오셔서 도가니수육 많이 드시더라고요, 간만에 도가니수육으로 몸...",-0.500000,0.548812,0.204487
35,명가옥,내장곰탕,0.5306,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...","[{'내장곰탕': ['내장 곰탕', '내장곰탕']}]",[내장곰탕 포장했는데 내장에서 잡내가 나요ㅜㅜ],-1.000000,0.301194,0.159814
40,명가옥,꼬리곰탕,0.3461,['구의동 명가옥 옥 관 이렇게 끝나는 식당들은 전 통과 노하우가 있어 굉장...,"[{'설렁탕': ['설렁탕']}, {'모듬수육': ['모듬수육', '모듬 수육']}...","[{'꼬리곰탕': ['꼬리곰탕', '꼬리 곰탕']}]",[메뉴가 비 싸혹시나 비싼값 할까했는데 맛도 우리 동네 설렁 탕 꼬리곰탕 보다 맛이...,-1.000000,0.301194,0.104243


In [129]:
data.to_csv("../data/senti_score_v2.0.0.csv",encoding='utf-8-sig')